<a href="https://colab.research.google.com/github/edalgoca/mod02_python/blob/main/20221028_Introducci%C3%B3n_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Instalar librerías**

In [ ]:
# Instalar librerias
!pip install pyspark
!pip install -q kaggle

# **Importar librerías**

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import zipfile
import os
import pandas as pd

ModuleNotFoundError: ignored

In [ ]:
import pandas as pd
df=pd.read_csv("test1.csv")
df

,Name,age,Experience,Salary
0,Krish,31,10,30000
1,Sudhanshu,30,8,25000
2,Sunny,29,4,20000
3,Paul,24,3,20000
4,Harsha,21,1,15000
5,Shubham,23,2,18000


# **Creación de una sesión con Spark**

In [ ]:
#Iniciar conexión de spark
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# **Conexión con kaggle para descargar los datos**
Para descargar datos desde Kaggle es necesario crear un token de acceso. Para esto debes seguir los siguientes pasos:

1.   Ingresa a tu perfil de kaggle
2.   Seleccionar *Account* y bajar hasta la opción *API*
3.   Dar click en *Create New API Token* y se descargará un archivo JSON con   tus credenciales
4.   Subir el archivo JSON a Colab Notebook


In [ ]:
# Creamos carpeta oculta en ambiente de linux sobre colab
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# Copiar el archivo JSON  a la carpeta oculta que creamos
!cp kaggle.json ~/.kaggle/

In [ ]:
# Cambiamos los permisos para permitir lectura de las credenciales
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Descargamos archivos indicando el usuario del propietario de los datos en kaggle y el nombre de dataset
! kaggle datasets download arevel/chess-games --force

100% 1.45G/1.45G [00:18<00:00, 84.3MB/s]
100% 1.45G/1.45G [00:18<00:00, 83.8MB/s]


In [ ]:
# Descomprimir el archivo
for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

# **Lectura de datos**

## Lectura de datos con Pandas

In [ ]:
df_pandas = pd.read_csv('/content/chess_games.csv')

## Lectura datos con PySpark

In [ ]:
df = spark.read.csv('/content/chess_games.csv', header=True)

# **Procesamiento de datos**

## Número de filas y columnas

pandas: 

```
df.shape
```



In [ ]:
print((df.count(), len(df.columns)))

(6256184, 15)


## Ver una muestra de los datos

pandas: 

```
df.head()
```

In [ ]:
df.show()

+------------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|             Event|          White|          Black|Result|   UTCDate| UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+------------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|        Classical |        eisaaaa|       HAMID449|   1-0|2016.06.30|22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|
|            Blitz |         go4jas|     Sergei1973|   0-1|2016.06.30|22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi...|      300+0|      Normal|1. e4 e5 2. b3 Nf

## Esquema de los datos

pandas: 

```
df.info()
```

In [ ]:
df.printSchema()

root
 |-- Event: string (nullable = true)
 |-- White: string (nullable = true)
 |-- Black: string (nullable = true)
 |-- Result: string (nullable = true)
 |-- UTCDate: string (nullable = true)
 |-- UTCTime: string (nullable = true)
 |-- WhiteElo: string (nullable = true)
 |-- BlackElo: string (nullable = true)
 |-- WhiteRatingDiff: string (nullable = true)
 |-- BlackRatingDiff: string (nullable = true)
 |-- ECO: string (nullable = true)
 |-- Opening: string (nullable = true)
 |-- TimeControl: string (nullable = true)
 |-- Termination: string (nullable = true)
 |-- AN: string (nullable = true)



## Estadística descriptiva 

pandas: 

```
df.describe()
```

In [ ]:
df.describe().show()

+-------+--------------------+--------------------+--------------------+-------+----------+--------+------------------+-----------------+------------------+--------------------+-------+--------------------+-----------+------------+--------------------+
|summary|               Event|               White|               Black| Result|   UTCDate| UTCTime|          WhiteElo|         BlackElo|   WhiteRatingDiff|     BlackRatingDiff|    ECO|             Opening|TimeControl| Termination|                  AN|
+-------+--------------------+--------------------+--------------------+-------+----------+--------+------------------+-----------------+------------------+--------------------+-------+--------------------+-----------+------------+--------------------+
|  count|             6256184|             6256184|             6256184|6256184|   6256184| 6256184|           6256184|          6256184|           6251516|             6251516|6256184|             6256184|    6256184|     6256184|          

## Filtrar el dataset 

pandas: 

```
df[df['Event']=='Classical']
```

In [ ]:
df.filter(df.Event == "Classical ").show()

+----------+---------------+----------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|     Event|          White|           Black|Result|   UTCDate| UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+----------+---------------+----------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|Classical |          Rokki|     solibalsara|   0-1|2016.07.02|23:16:20|    1868|    2123|           -4.0|            5.0|D08|Queen's Gambit Re...|     780+12|Time forfeit|1. d4 d5 2. c4 e5...|
|Classical |  DieAffeninsel|          AtitDJ|   0-1|2016.07.05|11:40:16|    1415|    1610|           -5.0|            6.0|A07|King's Indian Attack|     1200+0|      Normal|1. Nf3 d5 2. g3 N...|
|Classical |         Firhad|  

## Ordenar el dataset
pandas: 

```
df.sort_values(by=['UTCDate'], ascending=False)
```

In [ ]:
df.sort(F.col("UTCDate").desc()).show(10)

+--------------------+-------------+-------------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|               Event|        White|              Black|Result|   UTCDate| UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+--------------------+-------------+-------------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|   Blitz tournament |       iyei15|             SJ1992|   0-1|2016.07.31|00:00:03|    1288|    1449|           -7.0|            6.0|B30|Sicilian Defense:...|      180+0|Time forfeit|1. e4 { [%eval 0....|
| Classical tourna...|   Cullarvega|     Vlctor3million|   1-0|2016.07.31|00:00:03|    1753|    1727|           11.0|          -11.0|A44|  Old Benoni Defense|      600+0|      Norm

## Eliminar una columna:

pandas: 

```
df.drop(columns=['AN'], inplace=True)
```

In [ ]:
df = df.drop("AN")
df.show(10)

+------------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+
|             Event|          White|          Black|Result|   UTCDate| UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|
+------------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+
|        Classical |        eisaaaa|       HAMID449|   1-0|2016.06.30|22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|
|            Blitz |         go4jas|     Sergei1973|   0-1|2016.06.30|22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi...|      300+0|      Normal|
| Blitz tournament |Evangelistaizac|         kafune|   1-0|2016.06.30|22:00:02|    1647|    1688|   

## Eliminar datos nulos:

pandas: 

```
df.dropna(inplace=True)
```

In [ ]:
df = df.na.drop()
df.show()

+------------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+
|             Event|          White|          Black|Result|   UTCDate| UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|
+------------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+
|        Classical |        eisaaaa|       HAMID449|   1-0|2016.06.30|22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|
|            Blitz |         go4jas|     Sergei1973|   0-1|2016.06.30|22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi...|      300+0|      Normal|
| Blitz tournament |Evangelistaizac|         kafune|   1-0|2016.06.30|22:00:02|    1647|    1688|   

## Convertir datos a DataFrame de Pandas:

In [ ]:
df_Pandas = df.select("Event", "White", "Black", "Result").limit(50).toPandas()

In [ ]:
df_Pandas.head()

,Event,White,Black,Result
0,Classical,eisaaaa,HAMID449,1-0
1,Blitz,go4jas,Sergei1973,0-1
2,Blitz tournament,Evangelistaizac,kafune,1-0
3,Correspondence,Jvayne,Wsjvayne,1-0
4,Blitz tournament,kyoday,BrettDale,0-1


In [ ]:
df_Pandas.shape

(50, 4)

## UDF

Los UDF de Pyspark son similares a los métodos .map () y .apply () de pandas 'series y dataframes'.

Eliminar espacios en un string:

In [ ]:
UDF_python = F.udf(lambda s: s.replace(" ", ""), StringType())
df = df.withColumn("Event", UDF_python("Event"))

In [ ]:
df.show()

## Agrupar datos

pandas: 

```
df.groupby('Event').count()
```

In [ ]:
df_group = df.groupBy('Event').count()
df_group.show()

## Cambiar tipo de dato de una columna (date)

pandas: 

```
df['Date] = pd.to_datetime(df['UTCDate'], format="%Y/%m/%d")
```

In [ ]:
df = df.withColumn("Date", F.to_date(F.col("UTCDate"),"yyyy.MM.dd"))
df.printSchema()

## Renombrar una columna

pandas: 

```
df.rename(columns={'Opening':'Opening Type'}, inplace=True)
```

In [ ]:
df = df.withColumnRenamed("Opening", "Opening Type")

In [ ]:
df.show(3)

In [ ]:
Image(filename='sql.png')

# **Spark SQL en Python**

## Creación de vista del dataframe

In [ ]:
df.createOrReplaceTempView("chess_games")

## Cantidad de resgistros

In [ ]:
spark.sql("SELECT COUNT(*) FROM chess_games").show()

+--------+
|count(1)|
+--------+
| 6251516|
+--------+



## Valores únicos de una columna

In [ ]:
spark.sql("SELECT DISTINCT Result FROM chess_games").show()

+-------+
| Result|
+-------+
|1/2-1/2|
|    1-0|
|    0-1|
|      *|
+-------+



## Crear nueva columna

In [ ]:
spark.sql("SELECT Event, White, Black, Result, CASE WHEN Result = '1/2-1/2' THEN 'Empate' WHEN Result = '1-0' THEN 'White wins' WHEN Result = '0-1' THEN 'Black wins' ELSE 'NA' END AS definitive_results FROM chess_games").show()

+------------------+---------------+---------------+------+------------------+
|             Event|          White|          Black|Result|definitive_results|
+------------------+---------------+---------------+------+------------------+
|        Classical |        eisaaaa|       HAMID449|   1-0|        White wins|
|            Blitz |         go4jas|     Sergei1973|   0-1|        Black wins|
| Blitz tournament |Evangelistaizac|         kafune|   1-0|        White wins|
|   Correspondence |         Jvayne|       Wsjvayne|   1-0|        White wins|
| Blitz tournament |         kyoday|      BrettDale|   0-1|        Black wins|
| Blitz tournament | lucaseixasouza|         diguim|   0-1|        Black wins|
| Blitz tournament |       RENZZO77|HeadlessChicken|   0-1|        Black wins|
| Blitz tournament |          ipero|      Bayern123|   1-0|        White wins|
| Blitz tournament |Loginov19510410|        Kereshu|   0-1|        Black wins|
| Blitz tournament |     Shambobala|     cernunnoss|

## Calculos de una columna

In [ ]:
spark.sql("SELECT AVG(WhiteELO) as avg_white_elo, AVG(BlackELO) as avg_black_elo  FROM chess_games").show()

+------------------+----------------+
|     avg_white_elo|   avg_black_elo|
+------------------+----------------+
|1741.9246147654426|1740.37044838404|
+------------------+----------------+

